# Installing libraries

In [32]:
!pip install dash pandas plotly dash-bootstrap-components geopandas

In [33]:
!pip install contextily

In [34]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import geopandas as gpd

# Loading the CSV files and data preprocessing

In [35]:
# Load the CSV files
cholera_reported_cases = pd.read_csv('number_of_reported_cases.csv')
cholera_case_fr = pd.read_csv('implied_case_fatality_rate.csv')
cholera_reported_deaths = pd.read_csv('number_of_reported_deaths.csv')

# Convert columns to appropriate data types
cholera_reported_cases['Year'] = pd.to_numeric(cholera_reported_cases['Year'], errors='coerce')
cholera_reported_cases['Number of reported cases of cholera'] = pd.to_numeric(cholera_reported_cases['Number of reported cases of cholera'], errors='coerce')
cholera_reported_cases['Countries, territories and areas'] = cholera_reported_cases['Countries, territories and areas'].astype('category')

cholera_case_fr['Year'] = pd.to_numeric(cholera_case_fr['Year'], errors='coerce')
cholera_case_fr['Countries, territories and areas'] = cholera_case_fr['Countries, territories and areas'].astype('category')
cholera_case_fr['Cholera case fatality rate'] = pd.to_numeric(cholera_case_fr['Cholera case fatality rate'], errors='coerce')

cholera_reported_deaths['Year'] = pd.to_numeric(cholera_reported_deaths['Year'], errors='coerce')
cholera_reported_deaths['Countries, territories and areas'] = cholera_reported_deaths['Countries, territories and areas'].astype('category')
cholera_reported_deaths['Number of reported deaths from cholera'] = pd.to_numeric(cholera_reported_deaths['Number of reported deaths from cholera'], errors='coerce')

# Merge the datasets
merged_data = cholera_case_fr.merge(cholera_reported_deaths, on=["Year", "Countries, territories and areas"], how="outer")
merged_data = merged_data.merge(cholera_reported_cases, on=["Year", "Countries, territories and areas"], how="outer")


In [36]:
# Rearrange columns
rearranged_data = merged_data[[
    'Countries, territories and areas',
    'Number of reported cases of cholera',
    'Number of reported deaths from cholera',
    'Cholera case fatality rate',
    'Year'
]]

In [37]:
rearranged_data

,"Countries, territories and areas",Number of reported cases of cholera,Number of reported deaths from cholera,Cholera case fatality rate,Year
0,China,1.0,1.0,100.00,1949
1,Thailand,16.0,2.0,12.50,1949
2,Bangladesh,29809.0,12947.0,43.43,1950
3,Cambodia,10.0,1.0,10.00,1950
4,India,625.0,396.0,63.36,1950
...,...,...,...,...,...
2487,United Kingdom of Great Britain and Northern I...,19.0,0.0,0.00,2016
2488,United Republic of Tanzania,11360.0,172.0,1.50,2016
2489,United States of America,14.0,0.0,0.00,2016
2490,Yemen,15751.0,164.0,1.00,2016


In [38]:
int(rearranged_data['Year'].max())

2016

In [39]:
import base64

# Load the image and convert to base64
with open('cholera.png', 'rb') as img:
    encoded_image = base64.b64encode(img.read()).decode('utf-8')

In [40]:
!pip install python-docx

# Creating the app layout

In [41]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import base64
import docx

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Load the image and convert it to base64
with open('cholera.png', 'rb') as img:
    encoded_image = base64.b64encode(img.read()).decode('utf-8')

# Function to read content from a Word document
def read_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Load the homepage content from the uploaded Word document
homepage_content = read_docx('homepage.docx')

# Get unique countries and sort them alphabetically
sorted_countries = sorted(rearranged_data['Countries, territories and areas'].unique())

# Define layout
app.layout = dbc.Container(
    [
        # Logo section
        dbc.Row(
            dbc.Col(
                html.Div(
                    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '100%', 'text-align': 'center'}),
                    className="logo-container"
                ),
            ),
            justify='center',
        ),
        # Tabs for the dashboard
        dcc.Tabs(id="tabs", value='home', children=[
            dcc.Tab(label='Dashboard', value='home', children=[
                html.H2("Welcome to Cholera Compass!"),
                html.P(homepage_content),  # Use the content from the docx file
            ]),
            dcc.Tab(label='Plots by Country and Year', value='interactive_plots', children=[
                html.H2("Plots by Country and Year"),
                dcc.Graph(id="interactive_plot2"),
                dcc.Graph(id="interactive_graph3"),
                dcc.Graph(id="interactive_graph4"),
            ]),
            dcc.Tab(label='Incidents by Country and Year', value='country_calculation', children=[
                html.Label('Select a Country:'),
                dcc.Dropdown(id='selected_country', options=[
                    {'label': country, 'value': country} for country in sorted_countries
                ], value=sorted_countries[0]),  # Set the default value to the first country in the sorted list

                html.Label('Select Year Range:'),
                dcc.RangeSlider(
                    id='year_range',
                    min=int(rearranged_data['Year'].min()),
                    max=int(rearranged_data['Year'].max()),
                    value=[int(rearranged_data['Year'].min()), int(rearranged_data['Year'].max())],
                    marks={str(year): str(year) for year in range(int(rearranged_data['Year'].min()), int(rearranged_data['Year'].max()) + 1, 5)},  # Mark every 5 years
                    tooltip={"placement": "bottom", "always_visible": True}
                ),

                dcc.Tabs([
                    dcc.Tab(label="Charts", children=[
                        dcc.Graph(id="case_trend_chart"),
                        dcc.Graph(id="death_trend_chart"),
                        dcc.Graph(id="fatality_trend"),
                    ]),
                    dcc.Tab(label="Summary", children=[
                        html.Pre(id="country_report"),
                    ]),
                ]),
            ])
        ]),
    ],
    fluid=True,
)

# Call back functions for various graphs and charts

In [42]:
from dash import Input, Output, State

# Define the callback to update the summary report
@app.callback(
    Output('country_report', 'children'),
    Input('tabs', 'value'),  # Listen for tab changes
    Input('selected_country', 'value'),  # Selected country
    Input('year_range', 'value'),  # Selected year range
)
def update_summary(tab, selected_country, year_range):
    if tab == 'country_calculation':  # Trigger when the 'Summary' tab is clicked
        # Filter the data for the selected country and year range
        filtered_data = rearranged_data[
            (rearranged_data['Countries, territories and areas'] == selected_country) &
            (rearranged_data['Year'] >= year_range[0]) &
            (rearranged_data['Year'] <= year_range[1])
        ]

        # Check if filtered data exists
        if filtered_data.empty:
            return "No data available for the selected country and year range."

        # Calculate the total reported cases and deaths
        total_cases = filtered_data['Number of reported cases of cholera'].sum()
        total_deaths = filtered_data['Number of reported deaths from cholera'].sum()

        # Format the output summary
        summary = f"Country: {selected_country}\n" \
                  f"Year Range: {year_range[0]} - {year_range[1]}\n" \
                  f"Total Reported Cases: {total_cases}\n" \
                  f"Total Reported Deaths: {total_deaths}"

        return summary

    return ""  # Return empty when other tabs are clicked

In [43]:
from dash.dependencies import Input, Output

@app.callback(
    Output('interactive_plot2', 'figure'),
    Input('year_range', 'value')
)
def update_plot(year_range):
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) & (rearranged_data['Year'] <= year_range[1])]

    fig = px.bar(filtered_data, x='Year', y='Number of reported cases of cholera', color='Countries, territories and areas',
                 title="Global Comparison of Cholera Cases by Year")
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of Cases")

    return fig

# Similarly, define callbacks for other graphs


In [44]:
@app.callback(
    Output('interactive_graph3', 'figure'),  # Change output to the ID of the new graph
    Input('year_range', 'value')
)
def update_deaths_plot(year_range):
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) & (rearranged_data['Year'] <= year_range[1])]

    fig = px.bar(filtered_data, x='Year', y='Number of reported deaths from cholera', color='Countries, territories and areas',
                 title="Global Comparison of Cholera Deaths by Year")
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of Deaths")

    return fig

In [45]:
@app.callback(
    Output('interactive_graph4', 'figure'),  # Change output to the ID of the new graph
    Input('year_range', 'value')
)
def update_deaths_plot(year_range):
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) & (rearranged_data['Year'] <= year_range[1])]

    fig = px.bar(filtered_data, x='Year', y='Cholera case fatality rate', color='Countries, territories and areas',
                 title="Global Comparison of case fatality by Year")
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of Deaths")

    return fig

In [46]:
from dash.dependencies import Input, Output

@app.callback(
    Output('case_trend_chart', 'figure'),  # Output for case trend chart
    [Input('year_range', 'value'),  # Input for year range slider
     Input('selected_country', 'value')]  # Input for country dropdown selection
)
def update_case_trend_chart(year_range, selected_country):
    # Filter the data by the selected year range
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) &
                                    (rearranged_data['Year'] <= year_range[1])]

    # Further filter the data by the selected country
    if selected_country:
        filtered_data = filtered_data[filtered_data['Countries, territories and areas'] == selected_country]

    # Create a line chart to show the trend of cholera cases over the years
    fig = px.line(filtered_data, x='Year', y='Number of reported cases of cholera',
                  title=f"Cholera Cases Trend Over Time: {selected_country if selected_country else 'All Countries'}",
                  markers=True)  # Adds markers to each data point

    # Customize the layout
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of Reported Cases",
                      legend_title="Country", template="plotly")

    return fig

In [47]:
from dash.dependencies import Input, Output

@app.callback(
    Output('fatality_trend', 'figure'),  # Output for death trend chart
    [Input('year_range', 'value'),  # Input for year range slider
     Input('selected_country', 'value')]  # Input for country dropdown selection
)
def update_death_trend_chart(year_range, selected_country):
    # Filter the data by the selected year range
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) &
                                     (rearranged_data['Year'] <= year_range[1])]

    # Further filter the data by the selected country
    if selected_country:
        filtered_data = filtered_data[filtered_data['Countries, territories and areas'] == selected_country]

    # Create a line chart to show the trend of cholera deaths over the years
    fig = px.line(filtered_data, x='Year', y='Number of reported deaths from cholera',
                  title=f"Cholera Deaths Trend Over Time: {selected_country if selected_country else 'All Countries'}",
                  markers=True)  # Adds markers to each data point

    # Customize the layout
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of reported deaths from cholera",
                      legend_title="Country", template="plotly")

    return fig

In [48]:
from dash.dependencies import Input, Output

@app.callback(
    Output('death_trend_chart', 'figure'),  # Output for death trend chart
    [Input('year_range', 'value'),  # Input for year range slider
     Input('selected_country', 'value')]  # Input for country dropdown selection
)
def update_death_trend_chart(year_range, selected_country):
    # Filter the data by the selected year range
    filtered_data = rearranged_data[(rearranged_data['Year'] >= year_range[0]) &
                                     (rearranged_data['Year'] <= year_range[1])]

    # Further filter the data by the selected country
    if selected_country:
        filtered_data = filtered_data[filtered_data['Countries, territories and areas'] == selected_country]

    # Create a line chart to show the trend of cholera deaths over the years
    fig = px.line(filtered_data, x='Year', y='Cholera case fatality rate',
                  title=f"Cholera fatality rate Over Time: {selected_country if selected_country else 'All Countries'}",
                  markers=True)  # Adds markers to each data point

    # Customize the layout
    fig.update_layout(xaxis_title="Year", yaxis_title="Cholera case fatality rate",
                      legend_title="Country", template="plotly")

    return fig

# App Interface

In [49]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>